In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import sys
sys.path.append('../')

import shutil
from train import run
import os
import itertools

def write_errormsg2file(msg, error_file_name):
    if not os.path.isfile(error_file_name):
        with open(error_file_name, 'w') as f:
            f.write(f'error : {msg}\n')
    else:
        with open(error_file_name, 'a') as f:
            f.write(f'error : {msg}\n')

In [3]:
def do_exps(Ts, rotation_lambdas, lr_Hs, lambda_scale_factors, img_sizes, ht_init_methods, device, exp_dir = '../figs/test', save_special= False):
    exp_idx=0
    
    for ht_init_method in ht_init_methods:
        for img_size in img_sizes:
            for T in Ts:
                for rotation_lambda in rotation_lambdas:
                    for lr_H in lr_Hs:
                        for lambda_scale_factor in lambda_scale_factors:

                            exp_idx+=1

                            name= f'exp_idx({exp_idx})'
                            save_dir= f'{exp_dir}/ht_init_method({ht_init_method})@img_size({img_size})@T({T})@rotation_lambda({rotation_lambda})@lr_H({lr_H})@lambda_scale_factor({lambda_scale_factor})'

                            try:shutil.rmtree(save_dir)
                            except:pass

                            opts = [
                                'NAME', name,
                                'GENERAL.device', device,
                                'GENERAL.save_dir', save_dir,
                                'MODEL.MODEL_H.T', T,
                                'MODEL.MODEL_H.lr_H', lr_H,
                                'MODEL.MODEL_H.H_init', ht_init_method,
                                'MODEL.MODEL_A.scale_factor', lambda_scale_factor,
                                'MODEL.MODEL_A.rotation_lambda',rotation_lambda,
                                'DATASET.img_size', img_size 
                            ]


                            try:
                                run(opts= opts, save_special=save_special)
                            except:
                                error_file_name = f'{exp_dir}/errors.txt'
                                write_errormsg2file(f'ERROR : {save_dir}', error_file_name)
                                print(f'ERROR : {save_dir}')

In [4]:
device= 'cuda:0'

img_sizes= ['64']
Ts=['1', '2', '4', '8', '16', '32']
rotation_lambdas= ['10.0'] #['10.0', '100.0', '1000.0', '10000.0']
lr_Hs= ['1.0', '0.0']
lambda_scale_factors= ['1', '2', '3', '4']
ht_init_methods= ['hadamard_FourierBased']

do_exps(Ts, rotation_lambdas, lr_Hs, lambda_scale_factors, img_sizes, ht_init_methods, device, exp_dir = exp_dir, save_special= False)

# NEW

In [6]:
def do_exps(exps_dict= None, device= None, exp_dir = '../figs/test', save_special= False):    
    exp_idx = 0
    keys= list(exps.keys())
    
    val_list_list= []
    key_list = [] #eg: 'MODEL.MODEL_A.rotation_lambda'
    key_suffix_list= [] # eg: 'rotation_lambda'
    
    for key, val_list in exps_dict.items():
        key_list.append(key)
        key_suffix_list.append(key.split('.')[-1])
        
        val_list_list.append(val_list)
        
        
    attr_combination_list = [list(s) for s in itertools.product(*val_list_list)]
    
    for attr_combination in attr_combination_list:
        save_dir = f'{exp_dir}/'
        opts= []
        for idx in range(len(attr_combination)):
            opts += [key_list[idx], attr_combination[idx]]
            save_dir+= f'{key_suffix_list[idx]}({attr_combination[idx]})@'
    
        save_dir = save_dir[:-1] # remove last '@' 

        exp_idx+=1
        opts_other= ['NAME', f'exp_idx({exp_idx})', 
                     'GENERAL.device', device, 
                     'GENERAL.save_dir', save_dir]

        opts = opts_other + opts

        try:shutil.rmtree(save_dir)
        except:pass

        run(opts= opts, save_special=save_special)
        #try:
        #    run(opts= opts, save_special=save_special)
        #except:
        #    error_file_name = f'{exp_dir}/errors.txt'
        #    write_errormsg2file(f'ERROR : {save_dir}', error_file_name)
        #    print(f'ERROR : {save_dir}')                           

In [7]:
exp_dir= '../figs/test'
!rm -rf $exp_dir
!mkdir $exp_dir

In [ ]:
device = 'cuda:0'

exps = {
'MODEL.MODEL_H.T': ['1', '2', '4', '8', '16', '32'],
'MODEL.MODEL_H.lr_H': ['1.0','0.0'],
'MODEL.MODEL_H.H_init': ['hadamard_FourierBased'],
'MODEL.MODEL_A.lambda_scale_factor': ['1', '2', '3', '4'],
'MODEL.MODEL_A.rotation_lambda': ['10.0'],
'DATASET.img_size':  ['64']} 

do_exps(exps, device, exp_dir = exp_dir, save_special= False)